i used https://www.kaggle.com/abhishek/distilbert-use-features-oof kernel

In [1]:
!pip install ../input/sacremoses/sacremoses-master/ > /dev/null
import numpy as np
import pandas as pd
import os
import re
import gc
import sys
import glob
import torch
import math
sys.path.insert(0, "../input/transformers/transformers-master/")
import transformers

import tensorflow as tf
import tensorflow_hub as hub
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Input, Dropout, Lambda
from keras.optimizers import Adam
from keras.callbacks import Callback
from scipy.stats import spearmanr, rankdata
from urllib.parse import urlparse
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.linear_model import MultiTaskElasticNet

Using TensorFlow backend.


In [2]:
os.listdir("../input/google-quest-challenge")
root="../input/google-quest-challenge/"
train=pd.read_csv(root+"train.csv").fillna("none")
test=pd.read_csv(root+"test.csv").fillna("none")
test.columns

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host'],
      dtype='object')

In [3]:
targets=['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']
features=[c for c in test.columns]

>feature engneering

In [4]:
#get netloc
find=re.compile(r"^[^.]*")
train["netloc"] = train["url"].apply(lambda x: re.findall(find,urlparse(x).netloc)[0])
test['netloc'] = test['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])
#one hot encoder
cat_cols=["netloc","category"]
merged=pd.concat([train[cat_cols],test[cat_cols]])
ohe=OneHotEncoder().fit(merged)
ohe_train=ohe.transform(train[cat_cols]).toarray()
ohe_test=ohe.transform(test[cat_cols]).toarray()
ohe_train.shape

(6079, 64)

In [5]:
%%time
#embedding
module_url="../input/universalsentenceencoderlarge4/"
embed = hub.load(module_url)
em_cols=['question_title', 'question_body', 'answer']

embeddings_train = {}
embeddings_test = {}
for text in em_cols:
    print(text)
    train_text = train[text].str.replace('?', '.').str.replace('!', '.').tolist()
    test_text = test[text].str.replace('?', '.').str.replace('!', '.').tolist()
    
    curr_train_emb = []
    curr_test_emb = []
    batch_size = 4
    ind = 0
    while ind*batch_size < len(train_text):
        curr_train_emb.append(embed(train_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
        ind += 1
        
    ind = 0
    while ind*batch_size < len(test_text):
        curr_test_emb.append(embed(test_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
        ind += 1    
        
    embeddings_train[text + '_embedding'] = np.vstack(curr_train_emb)
    embeddings_test[text + '_embedding'] = np.vstack(curr_test_emb)
    
del embed
K.clear_session()
gc.collect()

question_title
question_body
answer
CPU times: user 57min 27s, sys: 3min 55s, total: 1h 1min 23s
Wall time: 18min 12s


302230

In [6]:
l2_dist = lambda x, y: np.power(x - y, 2).sum(axis=1)

cos_dist = lambda x, y: (x*y).sum(axis=1)

dist_features_train = np.array([
    l2_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
    l2_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
    l2_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding']),
    cos_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
    cos_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
    cos_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding'])
]).T

dist_features_test = np.array([
    l2_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding']),
    cos_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding'])
]).T

In [7]:
X_train=np.hstack([i for k,i in embeddings_train.items()]+[ohe_train,dist_features_train])
X_test=np.hstack([i for k,i in embeddings_test.items()]+[ohe_test,dist_features_test])
y_train=train[targets].values
print(X_train.shape)

(6079, 1606)


>model

In [8]:
# Compatible with tensorflow backend
class SpearmanRhoCallback(Callback):
    def __init__(self, training_data, validation_data, patience, model_name):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
        
        self.patience = patience
        self.value = -1
        self.bad_epochs = 0
        self.model_name = model_name

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_val = self.model.predict(self.x_val)
        rho_val = np.mean([spearmanr(self.y_val[:, ind], y_pred_val[:, ind] + np.random.normal(0, 1e-7, y_pred_val.shape[0])).correlation for ind in range(y_pred_val.shape[1])])
        if rho_val >= self.value:
            self.value = rho_val
        else:
            self.bad_epochs += 1
        if self.bad_epochs >= self.patience:
            print("Epoch %05d: early stopping Threshold" % epoch)
            self.model.stop_training = True
            #self.model.save_weights(self.model_name)
        print('\rval_spearman-rho: %s' % (str(round(rho_val, 4))), end=100*' '+'\n')
        return rho_val

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [9]:
def create_model():
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(512, activation='relu')(inputs)
    x = Dropout(0.2)(x)
    x=Dense(256,activation="relu")(x)
    x = Dense(y_train.shape[1], activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=x)
    model.compile(
        optimizer=Adam(lr=1e-4),
        loss=['binary_crossentropy']
    )
    model.summary()
    return model

In [10]:
all_pres=[]
kf=KFold(n_splits=5,shuffle=True)
for fold_,(tr_id,val_id) in enumerate(kf.split(X_train)):
    X_tr,y_tr = X_train[tr_id],y_train[tr_id]
    X_val,y_val = X_train[val_id],y_train[val_id]
    
    model=create_model()
    model.fit(X_tr,y_tr,epochs=100,batch_size=32,validation_data=(X_val,y_val),
              verbose=True,callbacks=[SpearmanRhoCallback(training_data=(X_tr, y_tr),validation_data=(X_val, y_val),
                                                          patience=5, model_name=f'best_model_batch{ind}.h5')])
    all_pres.append(model.predict(X_test))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1606)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               822784    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 30)                7710      
Total params: 961,822
Trainable params: 961,822
Non-trainable params: 0
_________________________________________________________________
Train on 4863 samples, validate on 1216 samples
Epoch 1/100
4863/4863 [==============================] - 2s 343us/step 

In [11]:
test_preds = np.array([np.array([rankdata(c) for c in p.T]).T for p in all_pres]).mean(axis=0)
max_val = test_preds.max() + 0.1
test_preds = test_preds/max_val + 1e-12

In [12]:
submission = pd.read_csv(root+'sample_submission.csv')
submission[targets] = test_preds
submission.to_csv("submission.csv", index = False)
submission.head(10)

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.605335,0.800252,0.963663,0.824617,0.067633,0.264650,0.695232,0.655325,0.414199,...,0.793531,0.508297,0.396975,0.496534,0.477631,0.472590,0.060491,0.101659,0.966604,0.706994
1,46,0.464188,0.461668,0.045369,0.141987,0.800672,0.714976,0.591052,0.530141,0.360849,...,0.521319,0.546944,0.354967,0.483092,0.463768,0.455787,0.938458,0.679269,0.077295,0.321781
2,70,0.616677,0.676749,0.464188,0.784709,0.822096,0.937198,0.603235,0.430162,0.812014,...,0.729259,0.502836,0.295316,0.653224,0.429742,0.366730,0.174333,0.269691,0.868725,0.464188
3,132,0.301197,0.079395,0.216341,0.420080,0.867465,0.847301,0.280613,0.489393,0.890149,...,0.276833,0.724218,0.817895,0.662046,0.754883,0.791850,0.604495,0.855283,0.756984,0.258349
4,200,0.944339,0.398656,0.590212,0.954421,0.509557,0.681790,0.878387,0.947280,0.846461,...,0.622558,0.846041,0.686410,0.906952,0.792271,0.689351,0.315060,0.738500,0.528040,0.859063
5,245,0.569208,0.890149,0.841840,0.162571,0.814535,0.482672,0.710355,0.455787,0.769586,...,0.794791,0.340685,0.211720,0.128965,0.145348,0.366730,0.141147,0.519639,0.876707,0.112161
6,257,0.393195,0.342365,0.388574,0.612896,0.527620,0.582651,0.189876,0.174753,0.632220,...,0.447385,0.868725,0.838479,0.683470,0.639782,0.786809,0.693972,0.860323,0.472170,0.510817
7,267,0.839740,0.704474,0.986767,0.178954,0.389414,0.100399,0.972485,0.974585,0.653644,...,0.888889,0.494014,0.710775,0.602815,0.551565,0.493594,0.013022,0.037807,0.993489,0.869565
8,284,0.275152,0.137366,0.399496,0.225163,0.527200,0.656165,0.342785,0.496954,0.800252,...,0.354127,0.659945,0.417559,0.522159,0.663726,0.697332,0.547784,0.763705,0.570048,0.466709
9,292,0.728838,0.566688,0.352867,0.814115,0.754883,0.743121,0.807813,0.738080,0.476371,...,0.302457,0.234825,0.286074,0.547784,0.351187,0.239866,0.341105,0.204999,0.853602,0.141147
